#### GISC 420 T1 2021
# Object orientation

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import math

We have seen how in Python more complicated data types have associated with them a variety of functions. For example `list`s have `sort()`, `append()` and `reverse()` functions. These are invoked on a variable of type `list` using the 'dot' notation

In [ ]:
x = [2, 3, 5, 7, 11, 13]
x.sort()
x.reverse()
x.append(17)
x

It is more accurate to call a variable of complex type like this an *object*. It turns out we can define our own object types or *classes* and use these when we are developing more complicated programs.

Spatial objects provide a nice example to illustrate the idea of *objected oriented programming* (OOP). OOP was the thing a few years back. It is now widely used, but not much discussed as such any more, because most modern programming languages provide support for the idea (in other words it is no longer a selling point, as it was when *Java* was the cool new object-oriented language). 

In this notebook, I will show how we can define our own object types in Python, and then show the advantages of the approach, by demonstrating some aspects of the `shapely.geometry` set of classes.

## The `class` keyword
Like functions, classes must be defined, this time using the `class` keyword. Here is a simple class definition for a `Point` object (by convention we capitalize the names of classes).

In [ ]:
class Point:
    """For dealing with points in 2D space"""

This definition is absolutely minimal. All it does it set aside a new class name `Point`. To use it we call `Point()` as if it were a function.

In [ ]:
p = Point()

And now we have a `Point` object.

In [ ]:
p

We can't do a whole lot with an empty point, so now let's assign some coordinates.

In [ ]:
p.x = 3.0
p.y = 4.0
p

We don't seem to have accomplished very much yet. But, for example, we can now do something like this.

In [ ]:
def distance(p1, p2):
    return math.sqrt((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2)

And now if we make another point object

In [ ]:
q = Point()
q.x = 7.0
q.y = 1.0

We have more 'human-readable' code:

In [ ]:
d = distance(p, q)
d

It's also worth knowing how to make a copy of an object. We could try the following

In [ ]:
r = p
r.x, r.y

So far so good, but turns out all we have done is to give a new name to the same data:

In [ ]:
p.x = 4.0
p.y = 3.0
r.x, r.y

To make a copy of the `Point` object `p` we use the `copy` module

In [ ]:
import copy
r = copy.copy(p)
p.x = 5.0
p.y = 12.0
p.x, p.y, r.x, r.y

So now we actually have two distinct `Point` object instances.

## Objects as more than data containers
All we have really seen so far is objects as a convenient way of moving perhaps more than one item of data around in a single 'container'. The object approach becomes much more powerful when we provide ways to manipulate objects that are specific to the things they represent. To do that, in addition to providing them with a name, we provide them with functions, called *methods* suitable for manipulating them in appropriate ways. This involves one more new keyword `self`.

In [ ]:
class Point:
    """Class to represent a point in 2D space"""
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)
        return None
    
    def __str__(self):
        return '(' + str(self.x) + ', ' + str(self.y) + ')'
    

And now when we `print` a `Point` object, it will use the `__str__` method of the class to format it.

In [ ]:
p = Point(3, 4)
print(p)

Now we are getting somewhere. The class definition has expanded to include an initialization method with the special name `__init__()` which expects two parameters `x` and `y`. It also has in the method signature an implied variable `self` which is a reference to the object itself, which must be included as a parameter in all the method definitions of the class.

This all seems like a bit of a nuisance, but already we are getting some subtle advantages. Notice how the initialization converts the internal attributes of the class `x` and `y` into `float` types. This means that it is now impossible to make a `Point` object with non-float coordinates.

In [ ]:
z = Point('a', 'b')

That in turn means that we can more confidently write functions that expect `Point` objects as parameters that won't cause unexpected errors.

We have also defined a method `__str__` which converts our `Point` object to a readable string format. That's how, when we use `print(p)` the `print()` function 'knows' how to print a representation of our object that is readable (it doesn't know... we told it how).

The `__something__()` methods are examples of built-in operations that we might want to perform on all kinds of objects. For example we might want to determine if two `Points` are equal.

In [ ]:
p = Point()
q = Point()
p == q

This is surprising since, if we print `p` and `q` they appear equal

In [ ]:
print(p)
print(q)

But we haven't told Python how it would know that two `Point` objects are equal. To do this we have to add a `__eq__()` method

In [ ]:
class Point:
    """Class to represent a point in 2D space"""
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)
        return None
    
    def __str__(self):
        return '(' + str(self.x) + ', ' + str(self.y) + ')'
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

In [ ]:
p = Point()
q = Point()
p == q

There are a whole host of built-in methods for any new class of objects we define, that we can define in this way if we want to make built-in operations like `>`, `<`, and so on, meaningful. It's not particularly obvious what it would mean for a point to be 'greater' than another, but even so it might be useful to be able to sort points (if we had them in a `list`). So here goes...

In [ ]:
class Point:
    """Class to represent a point in 2D space"""
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)
        return None
    
    def __str__(self):
        return str(self.x) + ', ' + str(self.y)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __gt__(self, other):
        return self.x + self.y > other.x + other.y


In [ ]:
import random

pts = []
for i in range(100):
    pts.append(Point(random.random(), random.random()))

pyplot.plot([p.x for p in pts], [p.y for p in pts])

In [ ]:
pts.sort()

pyplot.plot([p.x for p in pts], [p.y for p in pts])

This is subtle, but powerful stuff. The list `sort` function is making use of the way we have defined `__gt__` (greater than) to sort a list of points. Getting this kind of functionality without an ability to define classes would be really complicated.

## Adding our own functionality to a class
Providing standard operations like these to a newly defined class is certainly useful. Even better is providing application specific functions we need to solve our particular programming challenges. Take a look at the next cell.

In [ ]:
class Point:
    """Class to represent a point in 2D space"""
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)
        return None
    
    # __repr__ is similar to __str__
    # but should return a string that could be evaluated as code
    def __repr__(self):
        return 'Point(x=%s, y=%s)' % (self.x, self.y)
    
    # whereas __str__ emphasises human readability
    def __str__(self):
        return 'Point (%s, %s)' % (self.x, self.y)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __gt__(self, other):
        return self.x + self.y > other.x + other.y

    def distance(self, other):
        return math.sqrt((self.x - other.x) ** 2 + (self.y - other.y) ** 2)
    
    def manhattan_distance(self, other):
        return math.abs(self.x - other.x) + math.abs(self.y - other.y)
    
    def move(self, dx=0.0, dy=0.0):
        self.x += dx
        self.y += dy
        return self
    
    def distance_to_origin(self):
        return self.distance(Point())

Now we have a `Point` class we can do a fair amount with.

In [ ]:
p = Point(3, 4)
q = Point(5, 9)
r = Point(-5, 7)

In [ ]:
p.distance(q)

In [ ]:
q.distance(p)

In [ ]:
p.move(1, 2)
p

In [ ]:
p.distance_to_origin()

## Now let's make some other objects
We can make more complicated objects from simple one. A line segment is defined by two points, so let's define one of those:

In [ ]:
class Line_segment:
        '''A sraight line segment between two end points'''
        def __init__(self, p1, p2):
            self.p1 = p1
            self.p2 = p2
            return None
        
        def __repr__(self):
            return 'Line_segment(p1=%s, p2=%s)' % (repr(self.p1), repr(self.p2))
        
        def __str__(self):
            return 'Line segment from %s to %s' % (self.p1, self.p2)
        
        def length(self):
            return self.p1.distance(self.p2)
        
        def move(self, dx, dy):
            self.p1.move(dx, dy)
            self.p2.move(dx, dy)
            

A polygon can be defined by a series of points, so let's try that.

In [ ]:
class Polygon:
        '''A simple polygon object'''
        def __init__(self, pts):
            self.edges = []
            for p1, p2 in zip(pts[:-1], pts[1:]):
                self.edges.append(Line_segment(p1, p2))
            self.edges.append(Line_segment(pts[-1], pts[0]))
            return None
        
        def __repr__(self):
            pts = [edge.p1 for edge in self.edges]
            return 'Polygon(pts=%s)' % repr(pts)
        
        def __str__(self):
            return '\n'.join(['Polygon'] + [str(e) for e in self.edges])
        
        def perimeter(self):
            return sum([e.length() for e in self.edges])
        
        def num_sides(self):
            return len(self.edges)

And now let's do some things with those classes.

In [ ]:
A = Point()
B = Point(4, 0)
C = Point(4, 3)

AB = Line_segment(A, B)

ABC = Polygon([A, B, C])

In [ ]:
A, B, C

In [ ]:
AB, AB.length()

In [ ]:
ABC.perimeter()

Anyway... hopefully you get the general idea that we can build elaborate and complicated programs using objects, in such a way that *using* the code remains reasonably easy.

This approach underpins the complicated functionality we have already been accessing in this course in modules such as `geopandas` and `pyplot`. We don't much care how they do what they do, because we are working with the *public interface* of the classes defined by the module. In fact between versions of the modules, the project maintainers might change *how* the code works, but provided the *Application Programming Interface* (its **API**) remains unchanged, we can continue to use the code unaffected.

We'll take a look now at `shapely.geometry` to get some idea of this.

## The `shapely.geometry` API
The documentation for `shapely.geometry` is available [here](https://shapely.readthedocs.io/en/latest/). We are most interested in is here: [shapely.readthedocs.io/en/latest/manual.html#geometric-objects](https://shapely.readthedocs.io/en/latest/manual.html#geometric-objects).  Let's import this part of the module, also giving it an *alias* to make it less inconvenient to work with.

In [ ]:
import shapely.geometry

In [ ]:
A = shapely.geometry.Point(0, 0)

In [ ]:
print(A)

In [ ]:
B = shapely.geometry.Point(4, 0)
C = shapely.geometry.Point(4, 3)
print(B)
print(C)

Turns out that `shapely.geometry` can also draw `Point`s

In [ ]:
B

Although the drawings aren't very useful. 

Where `shapely.geometry` gets interesting is the geometric manipulations it provides. For example, even with points, we can do this

In [ ]:
A.buffer(1)

Lets' make a more complicated geometry (the module provides a host of these).

In [ ]:
ABC = shapely.geometry.LineString([A, B, C])
ABC

In [ ]:
poly_ABC = shapely.geometry.Polygon(ABC)
poly_ABC

In [ ]:
poly_ABC.buffer(1)

Another `shapely` module `affinity` provides methods for manipulating geometries in various ways

In [ ]:
import shapely.affinity

In [ ]:
polys = [poly_ABC]
for angle in range(45, 361, 45):
    polys.append(shapely.affinity.rotate(poly_ABC, angle, A))
rotated_triangles = shapely.geometry.GeometryCollection(polys)
rotated_triangles

You can also make `GeometryCollection` objects out of lists of geometries. This can be very handy (as will become clear in next week's lab assignment).

In [ ]:
rotated_triangles.buffer(0.35)

Have an explore of the documentation and see what other things you can do with this module!